<a href="https://colab.research.google.com/github/jonathannocek/yahoo-finance-stocks/blob/master/src/yahoo_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Yahoo Stock Data**

### **Part 1: Import packages**

In [2]:
import requests
import pandas as pd 
!pip install yahoo_fin
from yahoo_fin import stock_info as si
!pip install pandas_datareader
from pandas_datareader import DataReader
import numpy as np
from google.colab import drive
import datetime

# Mount Google Drive
drive.mount('drive')

# Constants
SP_500_TICKERS = si.tickers_sp500()
BASE_URL_1 = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
BASE_URL_2 = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=upgradeDowngradeHistory,recommendationTrend,' \
              'financialData,earningsHistory,earningsTrend,industryTrend&' \
              'corsDomain=finance.yahoo.com'

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


### **Part 2: Gather data**

This is gathering the data from the Yahoo Finance which rates the stock on a scale from 1 (Strong Buy) to 5 (Strong Sell). This data is released monthly by Yahoo Finance. The data below is one month behind the current month.

In [30]:
ratings_list = []
prices_list = []

for ticker in SP_500_TICKERS:
    # Get url and make request
    url = BASE_URL_1 + ticker + BASE_URL_2
    response = requests.get(url)
    response_json = response.json()

    # returns true if status_code less than 400
    if not response.ok:
      rating = 'N/A'
      price = 'N/A'    
    try:
      results_json = response_json['quoteSummary']['result'][0]
      rating = results_json['financialData']['recommendationMean']['fmt']
      price = results_json['financialData']['currentPrice']["fmt"]
    except:
      rating = 'N/A'
      price = 'N/A'
    
    ratings_list.append(rating)
    prices_list.append(price)

# Create pandas dataframe and insert data   
df = pd.DataFrame(list(zip(SP_500_TICKERS, prices_list, ratings_list)), columns =['Company','Price', 'Rating']) 
df = df.set_index('Company')

df.head().transpose()


Company,A,AAL,AAP,AAPL,ABBV
Price,98.30,12.16,156.11,497.48,94.86
Rating,2.40,3.30,2.30,2.20,1.90


#### **Get Current Month's Data**

In [68]:
data = []

for ticker in SP_500_TICKERS:
    # Get url and make request
    url = BASE_URL_1 + ticker + BASE_URL_2
    response = requests.get(url)
    response_json = response.json()

    try:
      # returns true if status_code less than 400
      if not response.ok:
        row = [ticker, None, None, None, None, None, None, None]

      # Parse Response
      results_json = response_json['quoteSummary']['result'][0]
      current_month = results_json['recommendationTrend']['trend'][0]

      # Get ratings
      strong_buy = current_month['strongBuy']
      buy = current_month['buy']
      hold = current_month['hold']
      sell = current_month['sell']
      strong_sell = current_month['strongSell']

      # Calculate average
      total_ratings = current_month['strongBuy'] + current_month['buy'] + current_month['hold'] + current_month['sell'] + current_month['strongSell']
      total_score = current_month['strongBuy'] * 1 + current_month['buy'] * 2 + current_month['hold'] * 3 + current_month['sell'] * 4 + current_month['strongSell'] * 5 
      average_score = round(total_score / total_ratings, 2)

      row = [ticker, average_score, strong_buy, buy, hold, sell, strong_sell, total_ratings]
      data.append(row)

    except Exception as e:
      row = [ticker, None, None, None, None, None, None, None]

# Create dataframe and insert data
my_columns = ['Company', 'Average Score', 'Strong Buy', 'Buy', "Hold", 'Sell', 'Strong Sell', 'Total Ratings']
df_current_month = pd.DataFrame(data, columns=my_columns) 
df_current_month = df_current_month.set_index('Company')



In [67]:
# Example of dataframe
df_current_month.transpose()

Company,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,...,USB,V,VAR,VFC,VLO,VMC,VNO,VRSK,VRSN,VRTX,VTR,VZ,WAB,WAT,WBA,WDC,WEC,WFC,WHR,WLTW,WM,WMB,WMT,WRB,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Average Score,1.71,2.39,2.56,1.87,2.3,2.36,1.7,2.0,2.24,1.84,2.15,3.0,2.78,1.87,2.67,2.41,2.33,2.93,2.47,2.69,2.67,2.44,2.5,2.5,1.36,1.86,2.3,2.0,1.86,1.81,2.68,1.64,2.48,2.18,1.73,1.79,2.16,2.88,2.05,2.63,...,2.79,1.77,2.5,2.48,2.43,1.73,2.17,2.4,3.25,1.81,3.1,2.69,2.23,2.4,2.17,2.0,2.45,2.61,2.67,2.14,2.33,1.95,2.38,3.18,2.06,2.14,3.05,2.29,2.41,2.46,2.72,3.0,2.08,2.11,2.19,2.32,2.07,2.09,2.35,2.0
Strong Buy,7.00,4.00,4.00,11.00,4.0,3.00,5.0,7.0,7.00,11.00,6.00,1.0,2.00,8.00,2.00,4.00,1.00,1.00,3.00,3.00,0.00,3.00,4.0,2.0,7.00,5.00,7.0,3.0,6.00,5.00,5.00,6.00,3.00,3.00,7.00,15.00,5.00,2.00,7.00,3.00,...,2.00,13.00,2.0,4.00,3.00,7.00,3.00,4.0,0.00,10.00,0.0,2.00,4.00,5.0,5.00,7.0,3.00,4.00,0.00,6.00,3.00,6.00,8.00,0.00,5.00,3.00,4.00,4.00,2.00,3.00,2.00,2.0,5.00,2.00,3.00,5.00,9.00,4.00,6.00,7.0
Buy,4.00,5.00,9.00,21.00,6.0,3.00,3.0,6.0,9.00,14.00,10.00,1.0,0.00,11.00,0.00,2.00,4.00,2.00,7.00,2.00,1.00,4.00,7.0,6.0,4.00,6.00,4.0,4.0,12.00,15.00,6.00,7.00,7.00,3.00,14.00,28.00,11.00,0.00,4.00,3.00,...,4.00,22.00,0.0,5.00,6.00,5.00,4.00,5.0,0.00,12.00,2.0,6.00,2.00,0.0,10.00,15.0,1.00,8.00,4.00,8.00,3.00,8.00,7.00,0.00,6.00,1.00,0.00,2.00,6.00,1.00,5.00,5.0,3.00,4.00,7.00,7.00,9.00,2.00,7.00,7.0
Hold,3.00,7.00,11.00,6.00,10.0,8.00,2.0,7.0,12.00,6.00,10.00,12.0,16.00,3.00,6.00,11.00,4.00,10.00,7.00,9.00,2.00,8.00,13.0,12.0,0.00,3.00,7.0,3.0,3.00,1.00,15.00,1.00,15.00,5.00,1.00,3.00,9.00,13.00,8.00,11.00,...,21.00,4.00,6.0,13.00,12.00,3.00,5.00,10.0,3.00,5.00,14.0,24.00,7.00,9.0,9.00,7.0,6.00,15.00,8.00,5.00,5.00,5.00,15.00,9.00,6.00,2.00,10.00,8.00,9.00,9.00,16.00,11.0,4.00,3.00,6.00,13.00,9.00,5.00,12.00,7.0
Sell,0.00,2.00,1.00,0.00,0.0,0.00,0.0,0.0,1.00,0.00,0.00,1.0,0.00,1.00,1.00,0.00,0.00,1.00,1.00,1.00,0.00,1.00,2.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,4.00,0.00,0.00,0.00,0.00,1.00,0.00,2.00,0.00,2.00,...,2.00,0.00,0.0,1.00,0.00,0.00,0.00,1.0,1.00,0.00,4.0,0.00,0.00,1.0,0.00,0.0,1.00,4.00,0.00,2.00,1.00,0.00,1.00,2.00,0.00,1.00,7.00,0.00,0.00,0.00,2.00,5.0,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.0
Strong Sell,0.00,0.00,2.00,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.00,1.0,0.00,0.00,0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.0,0.0,0.00,0.00,2.0,0.0,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,2.0,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.0
Total Ratings,14.00,18.00,27.00,38.00,20.0,14.00,10.0,20.0,29.00,31.00,26.00,16.0,18.00,23.00,9.00,17.00,9.00,15.00,19.00,16.00,3.00,16.00,26.0,20.0,11.00,14.00,20.0,10.0,21.00,21.00,31.00,14.00,25.00,11.00,22.00,47.00,25.00,17.00,19.00,19.00,...,29.00,39.00,8.0,23.00,21.00,15.00,12.00,20.0,4.00,27.00,20.0,32.00,13.00,15.0,24.00,29.0,11.00,31.00,12.00,21.00,12.00,19.00,32.00,11.00,17.00,7.00,22.00,14.00,17.00,13.00,25.00,25.0,13.00,9.00,16.00,25.00,28.00,11.00,26.00,21.0


**Basic Analysis: August 2020** 

Top 10 Stocks to Buy in August 2020 according to Yahoo Finance. 

Note: Some have small sample size - Total Ratings

In [70]:
# Get 10 stocks with the best ratings
df_current_month.sort_values(by="Average Score").head(10).transpose()

Company,ALGN,BIO,UNH,TMO,LKQ,GOOG,LDOS,DAL,AME,CMCSA
Average Score,1.36,1.5,1.5,1.53,1.57,1.6,1.6,1.61,1.64,1.65
Strong Buy,7.00,2.0,11.0,8.00,7.00,2.0,4.0,7.00,6.00,12.00
Buy,4.00,2.0,11.0,6.00,6.00,3.0,6.0,11.00,7.00,18.00
Hold,0.00,0.0,0.0,1.00,1.00,0.0,0.0,0.00,1.00,1.00
Sell,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00
Strong Sell,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.00
Total Ratings,11.00,4.0,22.0,15.00,14.00,5.0,10.0,18.00,14.00,31.00


In [71]:
# Get 10 stocks with the worst ratings 
df_current_month.sort_values(by="Average Score", ascending=False).head(10).transpose()

Company,CTVA,ED,UA,VRSN,CPB,UAA,WRB,PBCT,TWTR,BEN
Average Score,5.0,3.36,3.33,3.25,3.24,3.24,3.18,3.18,3.17,3.15
Strong Buy,0.0,1.00,0.00,0.00,0.00,2.00,0.00,0.00,2.00,1.00
Buy,0.0,0.00,0.00,0.00,2.00,2.00,0.00,0.00,2.00,0.00
Hold,0.0,7.00,4.00,3.00,10.00,19.00,9.00,9.00,21.00,8.00
Sell,0.0,5.00,2.00,1.00,4.00,6.00,2.00,2.00,10.00,4.00
Strong Sell,1.0,1.00,0.00,0.00,1.00,4.00,0.00,0.00,1.00,0.00
Total Ratings,1.0,14.00,6.00,4.00,17.00,33.00,11.00,11.00,36.00,13.00


### **Part 3: Save to CSV**

In [74]:
# Save files
now = datetime.datetime.now()
df.to_csv(f"yahoo_stock_data_{now.year}_{now.month}_{now.day}_old.csv")
df_current_month.to_csv(f"yahoo_stock_data_{now.year}_{now.month}_{now.day}_current.csv")
